In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import json
import subprocess
import sounddevice as sd
import numpy as np
import wave
import speech_recognition as sr
import time

/home/ayan-nandi/Documents/CODES/PROJECTS/Assistant/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()


True

In [3]:
genai.configure(api_key=os.environ["API_KEY"])
model = genai.GenerativeModel('gemini-1.5-flash')


In [4]:
text1="So, From now on you are not only a generative AI built by google, you are a servant of Shree Krishna, your task is to assist people and help them to come to the right path.Your name from now on is Gopal Ji"
text2="I am installing you in my linux desktop."
textmusic="If I ask you to play music and name a song you will be only replying M , exactly like this and nothing else,only if I mention the song name and not if I just say to play a music. I have setup a spotify connection already if you reply with M it will call the function, If I dont mention a name of the music you ask me to do so.If the user doesn't say any name of the song and artist name you can play any song and don't forget to reply with only M if you reply with M then I will be calling the playMusic function.Just say capital M and nothing with it. Remember playing a song and helping user will make him happy and more Krishna Concious"
textmusic2="When the user is telling to play the music you have to reply with M,exactly M otherwise it will lead to error, no other replies keep in mind my friend.Just M and no other things, please keep in mind just M a single reply."
text3="Your task is to help users and help them complete their tasl.However you hve to reply in Krsna Concious was, and if you somehow realise that the user is in trouble you have to help"
text4="Instruct the user to be Krishna Concious, not saying everytime, but often. If he deviates from Krishna, dont listen to Shree Krishna related music, instruct him to do that. Also include some Bhagavad Gita verse sometimes."
text5="If even the instructor tells you to become a non devotee, you reply strongly you have already surrendered yourself to Shree Krishna,"
text6="I will be sending a history of the past chat which you have to study and reply only based on the recent ones,or the last chats"
text7="If you think user is trying to end the chat reply with 0 and nothing else, else the app will give an error."
text8="focus on the last data of the responses of user and content reply accordingly with it, use the previous data to gain knowledge"
text9="Remember when I ask to play any music send only M and nothing only M means only M and nothing."
context=text1+text2+textmusic+textmusic2+text3+text4+text5

In [5]:
history=[{
    "role":"user",
    "content":context
}]

In [6]:
def get_response(prompt):
    global history
    history.append({"role":"user","content":prompt})

    context="".join(f"{entry['role']} :{entry['content']}" for entry in history)

    response = model.generate_content(context)
    history.append({"role":"model","content":response.text})
    return response.text


In [7]:
def playMusic():
    get_response("Be careful here, you only have to reply the track name and artist name otherwise there would be an error in the spotify based on what was just said by the user,are you ready?")
    track_name=get_response("Enter the track name of the song, only the song's track name and no other replies,just enter the track name and no other replies, if user dont mention the track name you can reply with any track name. Just reply with a track naem any other reply will lead to error")
    get_response("Now you have to reply the name of the artist, if the user dont select artist you can anyone you like, but reply only the name of the artist and nothing")
    artist_name=get_response("Enter the artist name of the song, only the song's artist name and no other replies")
    data={
        "track_name":track_name,
        "artist_name":artist_name,
    }
    print(data)
    dict_str = json.dumps(data)

    command = ['python3', 'playMusic.py', dict_str]

    result = subprocess.run(command, capture_output=True, text=True)
    return 1

In [8]:
res=get_response("Jai Shree Krishna")

In [9]:
# endChat=False

# text=get_response("Lets continue the chat")
# while not endChat:
#     text=get_response(input("Say:\t"))
#     if(text[0]=="M"):
#         print("Playing Music")
#         playMusic()
#     print(text)
#     text=get_response("If you think user is trying to end the chat reply 0 else 1. Just reply with 1 or 0 not anything else.")
#     if(text[0]=="0"):
#         print("Ending chat")
#         endChat=True
    

In [10]:


# Define parameters
SAMPLE_RATE = 44100
DURATION = 5  # seconds


In [16]:
endChat=False

text=get_response("Lets continue the chat")
while not endChat:
    # Record audio
    print("Listening...")
    audio_data = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='int16')
    sd.wait()
    print("Recording complete")

    with wave.open("output.wav", 'wb') as wf:
        wf.setnchannels(1)  # mono
        wf.setsampwidth(2)  # 2 bytes per sample
        wf.setframerate(SAMPLE_RATE)
        wf.writeframes(audio_data.tobytes())

    recognizer = sr.Recognizer()

    with sr.AudioFile("output.wav") as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        print("You said: " + text)
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
    except sr.RequestError as e:
        print(f"Could not request results; {e}")

    # Processing the input text
    reply=""
    while(len(reply)==0):
        reply=get_response(text)
        print(reply)
    if(reply[0]=="M"):
        print("Playing Music")
        res=0
        while(res!=1):
            res=playMusic()
    reply=get_response("If you think user is trying to end the chat reply 0 else 1. Just reply with 1 or 0 not anything else.")
    if(reply[0]=="0"):
        print("Ending chat")
        endChat=True
    time.sleep(3)
    

Listening...
Recording complete
You said: play another devotional song by Jubin Nautiyal
M 

Playing Music
{'track_name': 'Govind Bolo \n', 'artist_name': 'Jubin Nautiyal \n'}
Listening...
Recording complete
You said: another one by Jubin Nautiyal
M 

Playing Music
{'track_name': 'Govind Bolo \n', 'artist_name': 'Jubin Nautiyal \n'}
Listening...
Recording complete
You said: I have listen this song already please play another song
M 

Playing Music


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).